# Walmart Sales KPI Analysis & Executive Dashboard

## 1. Executive Summary

## Project Overview

This project analyzes historical Walmart sales data to identify key performance indicators (KPIs) and business drivers at the store and department level. Using Python and Plotly, the analysis transforms raw transactional data into an executive-ready, single-page dashboard that highlights sales trends, store performance, promotional impact, and external economic influences.

The goal of this notebook is not forecasting, but descriptive and diagnostic analytics: understanding what is happening and why, and presenting those insights in a clear, decision-oriented format.

## Business Questions

- This analysis is designed to answer the following questions:

- How do total and average weekly sales trend over time?

- How do holiday weeks compare to non-holiday weeks?

- How does store size relate to total sales and sales efficiency?

- How do different store types (A, B, C) perform?

- How prevalent are markdown promotions, and when do they occur?

- How do external factors (fuel price, CPI, unemployment) contextualize sales performance?

## Dataset Description

This project uses the public Walmart sales dataset originally released for a Kaggle forecasting competition.

### Data Sources

Train (train.csv)
- Weekly sales by store and department
- Grain: Store × Department × Week

Features (features.csv)
- Store-level weekly context including promotions, economic indicators, and weather
- Grain: Store × Week

Stores (stores.csv)
- Store metadata including type and physical size
- Grain: Store

The Weekly_Sales field represents observed historical sales and serves as the core metric for all KPIs.

## Data Modeling & Grain

The final analytical dataset preserves the original sales grain:

One row = one Store × one Department × one Week

Tables are joined using validated many-to-one relationships to avoid duplication and ensure KPI accuracy. Store-level and week-level attributes are safely attached to department-level sales records.

## Key Performance Indicators (KPIs)

The dashboard focuses on business-relevant KPIs, including:

- Total Sales

- Average Weekly Sales

- Holiday Sales Lift

- Sales per Square Foot

- Sales by Store Type

- Store Size vs Sales Relationship

- Promotion Prevalence (Markdown Weeks)

- External Context Indicators (Fuel Price, CPI, Unemployment)

These KPIs are calculated using explicit aggregation logic to ensure consistency across visualizations and downstream BI/Tableau tools.

## Visualization & Tools

- Python (pandas, numpy) for data preparation and KPI engineering

- Plotly for interactive, executive-style dashboard visualization

- Jupyter Notebook for reproducibility and narrative documentation

The final output is a single-page interactive dashboard suitable for portfolio presentation or replication in Power BI / Tableau.

## Scope & Limitations

- This analysis is descriptive and diagnostic; no predictive modeling is performed.

- Markdown effectiveness is explored at an aggregate level, not causal inference.

- Store size is used as a proxy for capacity, not exact square footage.

## How to Navigate This Notebook

**1)** Data loading and validation

**2)** Table joins and grain verification

**3)** Feature engineering and KPI construction

**4)** Aggregated KPI tables

**5)** Single-page executive dashboard

## 2. Data Loading

In [30]:
# standard libraries 
import os
import warnings
import numpy as np
import pandas as pd

# visualization tools 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# notebook settings 
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

# ---- Kaggle input file listing (optional; safe to remove outside Kaggle) ----
KAGGLE_INPUT_DIR = "/kaggle/input"
if os.path.exists(KAGGLE_INPUT_DIR):
    for dirname, _, filenames in os.walk(KAGGLE_INPUT_DIR):
        for filename in filenames:
            print(os.path.join(dirname, filename))

/kaggle/input/walmart-sales-forecast/features.csv
/kaggle/input/walmart-sales-forecast/stores.csv
/kaggle/input/walmart-sales-forecast/train.csv
/kaggle/input/walmart-sales-forecast/test.csv


In [31]:
# import CSV files

features_df = pd.read_csv("/kaggle/input/walmart-sales-forecast/features.csv")
stores_df = pd.read_csv("/kaggle/input/walmart-sales-forecast/stores.csv")
train_df= pd.read_csv("/kaggle/input/walmart-sales-forecast/train.csv")

## 3. Data Assessment

In [32]:
# assess features 
features_df.head(5)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [33]:
# assess stores
stores_df.head(5)

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [34]:
# assess training data
train_df.head(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


**For this project, the objective is to identify key performance indicators (KPIs) and build an executive dashboard. The analysis uses the training dataset as the source of historical sales data, which is combined with supporting feature and store metadata into a single analytical dataset. The test dataset is intentionally omitted, as the scope of this work is descriptive and diagnostic rather than predictive.** 

In [35]:
# validate data types in all three sets 
print('Train Dataframe')
print('')
print(train_df.dtypes)
print("Features Dataframe")
print('')
print(features_df.dtypes)
print("Store Dataframe")
print('')
print(stores_df.dtypes)

Train Dataframe

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
dtype: object
Features Dataframe

Store             int64
Date             object
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
IsHoliday          bool
dtype: object
Store Dataframe

Store     int64
Type     object
Size      int64
dtype: object


In [36]:
# convert date to datetime to all of joining 
train_df['Date'] = pd.to_datetime(train_df['Date'])
features_df['Date'] = pd.to_datetime(features_df['Date'])

In [37]:
# confirm conversion 
print('Train:', train_df['Date'].dtype)
print('Features:', features_df['Date'].dtype)


Train: datetime64[ns]
Features: datetime64[ns]


**Date was changed to datetime. This will allow each dataset to be joined.**

In [38]:
# standardize store and department types 

train_df['Store'] = train_df['Store'].astype('int64')
features_df['Store'] = features_df['Store'].astype('int64')
stores_df['Store'] = stores_df['Store'].astype('int64')

print('Train:', train_df['Store'].dtypes)
print('Features:', features_df['Store'].dtypes)
print('Store:', stores_df['Store'].dtypes)

Train: int64
Features: int64
Store: int64


**Before joining the datasets, the Store identifier is explicitly cast to the same data type (int64) across all tables. This ensures that join keys are consistent and prevents implicit type mismatches that can lead to failed joins, dropped records, or unintended row duplication.Although the Store column may already appear numeric, enforcing a consistent data type makes the join logic explicit and reproducible, which is especially important when working across multiple source files.The print statements are included as a validation step to confirm that all Store columns share the same data type prior to merging.**

## 4. Data Modeling and Joins

In [39]:
# join tables - join train + features on Store, Date

sale_features_df = train_df.merge(
        features_df,
    on=['Store', 'Date'],
    how = 'left',
    validate ='many_to_one',
    suffixes = ('','_feat')
)


In [40]:
# join stores on Store
final_df = sale_features_df.merge(
    stores_df,
    on = 'Store',
    how = 'left',
    validate = 'many_to_one'
)

In [41]:
# assess join
print("rows - train:", len(train_df))
print('rows - final:', len(final_df))
assert len(final_df) == len(train_df)

rows - train: 421570
rows - final: 421570


In [42]:
# assess missing values 
print(final_df.isna().mean().sort_values(ascending=False).head(12))

MarkDown2       0.736110
MarkDown4       0.679847
MarkDown3       0.674808
MarkDown1       0.642572
MarkDown5       0.640790
Store           0.000000
IsHoliday       0.000000
Dept            0.000000
Date            0.000000
Temperature     0.000000
Fuel_Price      0.000000
Weekly_Sales    0.000000
dtype: float64


In [43]:
# assess final dataframe
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Store           421570 non-null  int64         
 1   Dept            421570 non-null  int64         
 2   Date            421570 non-null  datetime64[ns]
 3   Weekly_Sales    421570 non-null  float64       
 4   IsHoliday       421570 non-null  bool          
 5   Temperature     421570 non-null  float64       
 6   Fuel_Price      421570 non-null  float64       
 7   MarkDown1       150681 non-null  float64       
 8   MarkDown2       111248 non-null  float64       
 9   MarkDown3       137091 non-null  float64       
 10  MarkDown4       134967 non-null  float64       
 11  MarkDown5       151432 non-null  float64       
 12  CPI             421570 non-null  float64       
 13  Unemployment    421570 non-null  float64       
 14  IsHoliday_feat  421570 non-null  boo

## Joining Sales, Features, and Store Metadata

The sales data (train_df) serves as the fact table and defines the analytical grain of the dataset: one row per Store × Department × Week. To enrich each sales record with contextual information, two left joins are performed.

First, the sales table is merged with the features table on Store and Date. The features table contains store-level, week-level information (e.g., promotions, economic indicators, and weather) and therefore has a one-to-many relationship with the sales data across departments. The validate='many_to_one' argument explicitly enforces this expected relationship and raises an error if duplicate feature records exist for a given store-week combination.

Second, the resulting dataset is merged with the store metadata table on Store. This table contains one record per store and adds structural attributes such as store type and size. The same many_to_one validation is applied to ensure each sales record maps to a single store record.

Left joins are used in both cases to preserve the full sales history, ensuring that no sales observations are dropped due to missing contextual data. Together, these joins produce a single analytical dataset that retains the original sales grain while adding all relevant business context for KPI calculation and dashboarding.

In [44]:
# assess IsHoliday vs. IsHoliday_feat

if 'IsHoliday_feat' in final_df.columns:
    mismatch = (final_df['IsHoliday'] != final_df['IsHoliday_feat']).mean()
    print("Holiday mismatch rate:", mismatch)


Holiday mismatch rate: 0.0


In [45]:
# mismatch rate is 0.0
# drop IsHoliday_feat column because it's a duplication 
final_df = final_df.drop(columns =['IsHoliday_feat'])
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Temperature   421570 non-null  float64       
 6   Fuel_Price    421570 non-null  float64       
 7   MarkDown1     150681 non-null  float64       
 8   MarkDown2     111248 non-null  float64       
 9   MarkDown3     137091 non-null  float64       
 10  MarkDown4     134967 non-null  float64       
 11  MarkDown5     151432 non-null  float64       
 12  CPI           421570 non-null  float64       
 13  Unemployment  421570 non-null  float64       
 14  Type          421570 non-null  object        
 15  Size          421

In [46]:
# final checks 
final_df.isnull().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday            0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
Type                 0
Size                 0
dtype: int64

In [47]:
# final checks 
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Temperature   421570 non-null  float64       
 6   Fuel_Price    421570 non-null  float64       
 7   MarkDown1     150681 non-null  float64       
 8   MarkDown2     111248 non-null  float64       
 9   MarkDown3     137091 non-null  float64       
 10  MarkDown4     134967 non-null  float64       
 11  MarkDown5     151432 non-null  float64       
 12  CPI           421570 non-null  float64       
 13  Unemployment  421570 non-null  float64       
 14  Type          421570 non-null  object        
 15  Size          421

## 5. Feature Engineering

In [48]:
# create date features for year, month, week and yearmonth
final_df['Year'] = final_df['Date'].dt.year
final_df['Month'] = final_df['Date'].dt.month
final_df['Week'] = final_df['Date'].dt.isocalendar().week
final_df['YearMonth'] = final_df['Date'].dt.to_period('M').astype(str)

# check features 
print('Year:',final_df['Year'].head(1))
print('Month:',final_df['Month'].head(1))
print('Week:',final_df['Week'].head(1))
print('YearMonth:',final_df['YearMonth'].head(1))

Year: 0    2010
Name: Year, dtype: int32
Month: 0    2
Name: Month, dtype: int32
Week: 0    5
Name: Week, dtype: UInt32
YearMonth: 0    2010-02
Name: YearMonth, dtype: object


In [49]:
# create holiday flag
final_df['Holiday_Flag'] = final_df['IsHoliday'].astype(int)

In [50]:
# create markdown column where markdowns are present 
markdown_cols = ['MarkDown1',
                 'MarkDown2',
                 'MarkDown3',
                 'MarkDown4',
                 'MarkDown5']
final_df['Any_Markdown'] = final_df[markdown_cols].gt(0).any(axis=1).astype(int)


In [51]:
# total mark downs 
final_df['Total_Markdown'] = final_df[markdown_cols].fillna(0).sum(axis=1)
print(final_df['Total_Markdown'].head(3))


0    0.0
1    0.0
2    0.0
Name: Total_Markdown, dtype: float64


## Feature Engineering for Time, Holidays, and Promotions

To support KPI aggregation and interactive dashboarding, several helper features are derived from the raw data. These features simplify downstream grouping logic and ensure consistent calculations across visualizations.

**Date-Based Features**

The Date column is decomposed into multiple time components:

Year and Month enable annual and monthly trend analysis.

Week (ISO week) supports weekly aggregation while preserving calendar consistency.

YearMonth provides a compact, sortable period identifier that is well-suited for plotting and BI tools.

These features allow flexible temporal aggregation without repeatedly parsing the original datetime field.

Validation prints are included to confirm that each derived field is populated correctly.

**Holiday Indicator**

A binary Holiday_Flag is created from the original IsHoliday column. Representing holidays as a numeric flag simplifies aggregation and comparison logic when computing KPIs such as holiday sales lift.

**Promotion (Markdown) Features**

Markdown activity is captured through two derived features:

Any_Markdown
A binary indicator that flags whether any markdown was active during a given store-week. This is useful for measuring the prevalence of promotions and comparing promotional versus non-promotional periods.

Total_Markdown
The sum of all markdown values (MarkDown1–MarkDown5) for each observation. Missing markdown values are treated as zero to ensure accurate aggregation.

Together, these features allow the analysis to quantify both the presence and intensity of promotional activity and relate it to sales performance.

**Why This Matters**

These engineered features:

- Reduce repetitive logic in visualizations

- Enable clean KPI calculations

- Improve performance and readability in Plotly and Power BI / Tableau

- Preserve the original sales grain while adding analytical flexibility

## 6. KPI Aggregation & Analytical Views

In [52]:
# create weekly sales for a baseline 
weekly_sales = (
    final_df
    .groupby('Date', as_index=False)['Weekly_Sales']
    .sum()
)

In [53]:
# create monthly sales
monthly_sales = (
    final_df
    .groupby('YearMonth', as_index = False)['Weekly_Sales']
    .sum()
)

In [54]:
# create store performance 
store_perf = (
    final_df
    .groupby(['Store', 'Type'], as_index=False)
    .agg(
        Total_Sales=('Weekly_Sales', 'sum'),
        Avg_Weekly_Sales=('Weekly_Sales', 'mean'),
        Store_Size=('Size', 'first')
    )
)


In [55]:
# create store peformance efficiency 
store_perf['Sales_per_SqFt'] = (
    store_perf['Total_Sales'] / store_perf['Store_Size']
)


In [56]:
# validate KPI logic 
final_df[['Weekly_Sales','Store','Dept']].describe()
store_perf.sort_values('Total_Sales', ascending=False).head()


,Store,Type,Total_Sales,Avg_Weekly_Sales,Store_Size,Sales_per_SqFt
19,20,A,3.013978e+08,29508.301592,203742,1479.311053
3,4,A,2.995440e+08,29161.210415,205863,1455.064550
13,14,A,2.889999e+08,28784.851727,200898,1438.540510
12,13,A,2.865177e+08,27355.136891,219622,1304.594730
1,2,A,2.753824e+08,26898.070031,202307,1361.210640


To support dashboard visualizations and ensure consistent KPI calculations, the data is aggregated into multiple analytical views at different levels of detail. These aggregated tables separate raw data preparation from business-level reporting and improve both performance and interpretability.

**Weekly Sales (Baseline Trend)**

Weekly sales are aggregated across all stores and departments to establish a baseline time series. This view supports trend analysis and is used for executive-level monitoring of overall sales performance.

**Monthly Sales (Executive Summary View)**

Sales are further aggregated at the month level using the derived YearMonth field. Monthly aggregation smooths week-to-week variability and provides a higher-level view suitable for strategic reporting and dashboard summaries.

**Store Performance Metrics**

Store-level KPIs are calculated by aggregating sales across time and departments for each store and store type. This view captures both total performance and average weekly behavior while preserving store attributes such as size. These metrics enable comparisons across stores and support efficiency measures such as sales per square foot.

**KPI Validation**

Summary statistics and sorted views are used to validate aggregation logic and ensure that KPI calculations behave as expected. This step provides a quick sanity check before the results are used in visualizations or downstream BI tools.

## 7. Plotly Single Page Dashboard

In [57]:
# aggregations
weekly_sales = (
    final_df
    .groupby(['Date', 'IsHoliday'], as_index=False)
    .agg(Weekly_Sales=('Weekly_Sales', 'sum'))
)

weekly_all = (
    weekly_sales
    .groupby('Date', as_index=False)
    .agg(Weekly_Sales=('Weekly_Sales', 'sum'))
)

monthly_sales = (
    final_df
    .groupby('YearMonth', as_index=False)
    .agg(Monthly_Sales=('Weekly_Sales', 'sum'))
)

type_sales = (
    final_df
    .groupby('Type', as_index=False)
    .agg(Total_Sales=('Weekly_Sales', 'sum'))
)

store_perf = (
    final_df
    .groupby(['Store', 'Type'], as_index=False)
    .agg(
        Total_Sales=('Weekly_Sales', 'sum'),
        Avg_Weekly_Sales=('Weekly_Sales', 'mean'),
        Store_Size=('Size', 'first')
    )
)
store_perf['Sales_per_SqFt'] = store_perf['Total_Sales'] / store_perf['Store_Size']

# kpi calculations
total_sales = final_df['Weekly_Sales'].sum()
avg_weekly_sales = weekly_all['Weekly_Sales'].mean()

holiday_week_avg = weekly_sales.loc[weekly_sales['IsHoliday'] == True, 'Weekly_Sales'].mean()
nonholiday_week_avg = weekly_sales.loc[weekly_sales['IsHoliday'] == False, 'Weekly_Sales'].mean()
holiday_lift_pct = ((holiday_week_avg / nonholiday_week_avg) - 1) * 100 if nonholiday_week_avg else 0

sales_per_sqft = store_perf['Total_Sales'].sum() / store_perf['Store_Size'].sum()

promo_weeks_pct = final_df.groupby(['Store', 'Date'])['Any_Markdown'].max().mean() * 100
avg_markdown = final_df.loc[final_df['Total_Markdown'] > 0, 'Total_Markdown'].mean()
avg_markdown = 0 if np.isnan(avg_markdown) else avg_markdown

avg_fuel = final_df['Fuel_Price'].mean()
avg_unemp = final_df['Unemployment'].mean()


# sort for visuals
monthly_sales_sorted = monthly_sales.copy()
monthly_sales_sorted['YearMonth_dt'] = pd.to_datetime(monthly_sales_sorted['YearMonth'] + "-01")
monthly_sales_sorted = monthly_sales_sorted.sort_values('YearMonth_dt')

type_sales_sorted = type_sales.sort_values('Total_Sales', ascending=False)

# build dashboard layout
fig = make_subplots(
    rows=6, cols=4,
    specs=[
        [{"type":"indicator","colspan":2}, None, {"type":"indicator","colspan":2}, None],
        [{"type":"indicator","colspan":2}, None, {"type":"indicator","colspan":2}, None],
        [{"type":"indicator","colspan":2}, None, {"type":"indicator","colspan":2}, None],
        [{"type":"indicator","colspan":2}, None, {"type":"indicator","colspan":2}, None],
        [{"type":"xy","colspan":2}, None, {"type":"xy","colspan":2}, None],
        [{"type":"xy","colspan":2}, None, {"type":"xy","colspan":2}, None],
    ],
    row_heights=[0.10, 0.10, 0.10, 0.10, 0.30, 0.30],
    column_widths=[0.25, 0.25, 0.25, 0.25],
    horizontal_spacing=0.06,
    vertical_spacing=0.06,
    subplot_titles=(
        "", "", "", "",
        "", "", "", "",
        "", "", "", "",
        "", "", "", "",
        "Weekly Sales Trend", "Monthly Sales Trend",
        "Total Sales by Store Type", "Store Size vs Total Sales"
    )
)

# indicator stylying 
IND_TITLE = {"font": {"size": 16}}
IND_NUM_INT = {"valueformat": ",.0f", "font": {"size": 38}}
IND_NUM_DEC = {"valueformat": ".2f",  "font": {"size": 38}}
IND_NUM_PCT = {"valueformat": ".2f",  "font": {"size": 38}}


# row 1: Total Sales | Avg Weekly Sales
fig.add_trace(go.Indicator(
    mode="number",
    value=total_sales,
    title={"text":"Total Sales", **IND_TITLE},
    number=IND_NUM_INT
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=avg_weekly_sales,
    title={"text":"Avg Weekly Sales", **IND_TITLE},
    number=IND_NUM_INT
), row=1, col=3)

# row 2 Holiday Lift | Sales per Sq Ft
fig.add_trace(go.Indicator(
    mode="number",
    value=holiday_lift_pct,
    title={"text":"Holiday Lift (%)", **IND_TITLE},
    number=IND_NUM_PCT
), row=2, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=sales_per_sqft,
    title={"text":"Sales per Sq Ft", **IND_TITLE},
    number=IND_NUM_DEC
), row=2, col=3)

# row 3: Store-Weeks w/ Markdown | Avg Markdown
fig.add_trace(go.Indicator(
    mode="number",
    value=promo_weeks_pct,
    title={"text":"Store-Weeks w/ Markdown (%)", **IND_TITLE},
    number=IND_NUM_PCT
), row=3, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=avg_markdown,
    title={"text":"Avg Markdown (when > 0)", **IND_TITLE},
    number=IND_NUM_INT
), row=3, col=3)

# row 4: Avg Fuel Price | Avg Unemployment
fig.add_trace(go.Indicator(
    mode="number",
    value=avg_fuel,
    title={"text":"Avg Fuel Price", **IND_TITLE},
    number=IND_NUM_DEC
), row=4, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=avg_unemp,
    title={"text":"Avg Unemployment", **IND_TITLE},
    number=IND_NUM_DEC
), row=4, col=3)

fig.update_traces(selector=dict(type="indicator"), title_align="center", align="center")

# charts 
fig.add_trace(go.Scatter(
    x=weekly_all['Date'],
    y=weekly_all['Weekly_Sales'],
    mode="lines",
    name="Weekly Sales"
), row=5, col=1)

fig.add_trace(go.Scatter(
    x=monthly_sales_sorted['YearMonth'],
    y=monthly_sales_sorted['Monthly_Sales'],
    mode="lines",
    name="Monthly Sales"
), row=5, col=3)

fig.add_trace(go.Bar(
    x=type_sales_sorted['Type'],
    y=type_sales_sorted['Total_Sales'],
    name="Sales by Type"
), row=6, col=1)

fig.add_trace(go.Scatter(
    x=store_perf['Store_Size'],
    y=store_perf['Total_Sales'],
    mode="markers",
    text=store_perf['Store'],
    name="Stores",
    hovertemplate=(
        "Store: %{text}<br>"
        "Type: %{customdata[0]}<br>"
        "Size: %{x}<br>"
        "Total Sales: %{y:,.0f}<br>"
        "Avg Weekly Sales: %{customdata[1]:,.0f}<br>"
        "Sales/SqFt: %{customdata[2]:.2f}<extra></extra>"
    ),
    customdata=np.column_stack([
        store_perf['Type'],
        store_perf['Avg_Weekly_Sales'],
        store_perf['Sales_per_SqFt']
    ])
), row=6, col=3)

# layout stylying 
fig.update_layout(
    title="Walmart Sales Executive Dashboard (Single-Page)",
    height=1200,
    showlegend=False,
    margin=dict(l=40, r=40, t=90, b=40),
    template="plotly_white"
)

fig.update_annotations(font_size=14)

# axis labels
fig.update_xaxes(title_text="Date", row=5, col=1)
fig.update_yaxes(title_text="Sales", row=5, col=1)

fig.update_xaxes(title_text="Year-Month", row=5, col=3)
fig.update_yaxes(title_text="Sales", row=5, col=3)

fig.update_xaxes(title_text="Store Type", row=6, col=1)
fig.update_yaxes(title_text="Sales", row=6, col=1)

fig.update_xaxes(title_text="Store Size", row=6, col=3)
fig.update_yaxes(title_text="Total Sales", row=6, col=3)

fig.show()
fig.write_html("walmart_executive_dashboard_offline.html", include_plotlyjs=True)

In [58]:
# export to CSV to use to make dashboard 
final_df.to_csv("walmart_fact_table.csv", index=False)


## 8. Key Insights

**Holiday periods drive higher sales performance.**
Weekly sales during holiday weeks exceed non-holiday weeks, confirming that seasonality plays a significant role in revenue generation and should remain a focal point for planning and inventory alignment.

**Store size correlates with total sales, but not efficiency.**
Larger stores generally produce higher total sales; however, sales per square foot varies by store type, indicating that physical size alone does not determine operational efficiency.

**Store type materially impacts performance.**
Sales contribution differs across store types (A, B, C), suggesting that format and location characteristics influence revenue potential and should be considered in benchmarking and strategic decision-making.

**Markdown activity is common but uneven.**
A meaningful percentage of store-weeks include markdowns, highlighting promotions as a frequently used pricing lever. The wide range in markdown magnitude suggests inconsistent promotional intensity across stores and time periods.

**External economic factors provide important context.**
Average fuel prices and unemployment levels vary over the analysis period and help contextualize sales trends, reinforcing the importance of interpreting performance alongside broader economic conditions.

## 9. Next Steps 

- **Deploy the dashboard in a business intelligence tool.**
Recreate the single-page executive dashboard in Power BI or Tableau using the exported analytical dataset. This would enable scheduled refreshes, stakeholder access, and integration with existing reporting workflows.

- **Enable ongoing performance monitoring.**
Establish automated data refresh and KPI tracking to support continuous monitoring of sales trends, holiday performance, promotional activity, and store efficiency.

- **Evaluate promotional effectiveness.**
Extend the analysis to estimate the incremental impact of markdowns on sales, helping differentiate between promotional lift and baseline demand.

- **Incorporate forecasting and scenario analysis.**
Build time-series or machine learning models to forecast weekly sales and assess potential outcomes under different promotional or economic scenarios.

- **Segment and benchmark stores.**
Group stores by size, type, and efficiency (e.g., sales per square foot) to identify best-performing segments and operational improvement opportunities.